In [1]:
import requests
from urllib import request
from bs4 import BeautifulSoup
import time
import pandas as pd

In [4]:
ACODE = {"호텔신라":"008770","두산퓨얼셀":"336260","한화솔루션":"009830",
            "에코프로":"086520","신성이엔지":"011930","유니슨":"018000",
            "카카오":"035720","네이버":"035420","엔씨소프트":"036570",
            "sds":"018260","삼성전자":"005930","SK하이닉스":"000660",
            "skc":"011790","한솔케미칼":"014680","DB하이텍":"000990",
            "롯데쇼핑":"023530","신세계":"004170","삼성물산":"028260",
            "동서":"026960","kb금융":"105560","한국금융지주":"071050","신한지주":"055550",
            "우리금융지주":"316140","하나금융지주":"086790",
         "넥슨지티":"041140","한빛소프트":"047080","코세스":"089890",
      "드래곤플라이":"030350","에이트원":"230980","엔텔스":"069410"}


In [3]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
DATE_STOP = '2021-08-19'


def title_crawl(ls, ticker, result):
    global DATE_STOP

    for i in range(0,len(ls)-1):
        response = requests.get('https://gall.dcinside.com' + str(ls[i]),headers = HEADERS)
        soup = BeautifulSoup(response.content,'html.parser')
        items = soup.find_all("table",{'class':'gall_list'})
        contents = soup.find('tbody').find_all('tr')
      
        for j in contents:         
            if j.find('td',{'class':'gall_writer ub-writer'}).text=='운영자':
                pass
            else:
                # 날짜
                date_dict = j.find('td',{'class':'gall_date'}).attrs

                if date_dict['title'][:10] <= DATE_STOP: 
                    return -1
                else:
                    # 제목
                    title = j.find('a').text
                    # 추천수
                    recommend_tag = j.find('td', class_='gall_recommend')
                    recommend = recommend_tag.text

                    # 조회수
                    views_tag = j.find('td', class_='gall_count')
                    views = views_tag.text

                    put_data = {
                        'code': ACODE[ticker],
                        'date' : date_dict['title'],
                        'title' : title,
                        'view' : views,
                        'recommend' : recommend
                    }

                    RESULT.append(put_data)
                   

In [5]:
def DC_crawling(ticker, page):
    RESULT = []
    url = f"https://gall.dcinside.com/board/lists?id=neostock&s_type=search_subject_memo&s_keyword={ticker}"

    a = f'/board/lists?id=neostock&s_type=search_subject_memo&s_keyword={ticker}'
    response = requests.get(url,headers=HEADERS)
    soup = BeautifulSoup(response.text,'html.parser')
    items = soup.find('div',{'class':"bottom_paging_box"})

    #리스트 만들기 url
    url_list=[a]
    for i in items.find_all('a'):
        url_url = i['href']
        url_list.append(url_url)

    #  첫페이지
    title_crawl(url_list, ticker, RESULT)
    
    k = 0
    while k < page :
        response = requests.get('https://gall.dcinside.com' + str(url_list[-1]),headers=HEADERS)
        soup = BeautifulSoup(response.content,'html.parser')
        items = soup.find('div',{'class':"bottom_paging_box"})
        url_list=[str(url_list[-1])]

        for i in items.find_all('a'):
            url_url = i['href']
            url_list.append(url_url)
        url_list.pop(1) 
        
        if title_crawl(url_list, ticker,RESULT) == -1:
            break
        else:
            k += 1

    return RESULT

In [6]:
import time
import random
import datetime
from kafka import KafkaProducer

bootstrap_servers = ['localhost:9092','localhost:9093']  # kafka broker ip
topicName = 'DC'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

for t in ACODE.keys():
        abc = DC_crawling(t, 100)

for i in abc:
    
#print(page_check(url_list))
    print(i)
    producer.send(topicName, str(i).encode())

        #text = 'This is ' + str(i) + ' DC'
        #print(text)
    tim = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            #producer.send(topicName, text.encode())
    producer.send(topicName, tim.encode())

    time.sleep(1)

NoBrokersAvailable: NoBrokersAvailable

In [ ]:
if __name__ == "__main__":
    for t in ACODE.keys():
        DC_crawling(t, 100)